## NOTEBOOK: Profile a ResNet18 Model

This tutorial validates, compiles, and profiles a ResNet18 model for inference on Envise using the Idiom Software Stack.

Run this Jupyter notebook on an environment that has a GPU instance. 

The model will traverse through the following stages in the developer flow:

**Validate the model**
    
    The operator coverage tool is invoked at this stage and checks for supported and unsupported ONXX operators in the model. 

**Compile the model** 
    
    The compile() Idiom API is invoked at this stage and the ONNX model is compiled for Envise.

**Profile the model** 
    
    The profile() Idiom API is invoked at this stage and the model is executed at runtime in an Envise-simulated environment for performance metrics.

**SYSTEM COMPONENT MINIMUM REQUIREMENTS**

* CPU: Any X86-64 architecture with 4 cores
* RAM: 64 GB memory
* GPU: One Nvidia 2080

#### Set up Imports

In [ ]:
import idiom
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import itertools

#### Export file to ONNX
Specify the `batch_size` and use the variable where ever the batch size is required.

In [ ]:
import torchvision.models as models
model = models.resnet18(pretrained=False, num_classes=10)
model.load_state_dict(torch.load('/idiom-eap/examples/00-getting-started/resnet-18-imagewoof2-320.pth'))
model.eval()
onnx_file_name = 'rn18.onnx'
batch_size  = 2
# Input to the model with the batch_size
x = torch.randn(batch_size, 3, 320, 320, requires_grad=True)

torch.onnx.export(model,                  # model being run
                  x,                      # model input (or a tuple for multiple inputs)
                  onnx_file_name,         # where to save the model (can be a file or file-like object)
                  verbose=False,
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'],) # the model's output names


#### Validate Model

The model needs to get validated for Operator Coverage. Here the ONNX model is scanned and you get an output that shows a list of supported and unsupported operations by the compiler.

The ONNX file that is being validated is: `resnet-18-imagewoof2-320.onnx`

The ``idiom.cc.onnx.check_op_cov`` API command invokes the Operator Coverage functionality. It accepts one argument: an ONNX model.
 

In [ ]:
from idiom.cc.onnx import check_op_cov
check_op_cov(onnx_file_name)

Above, you can see that the model is fully supported. Thus, it is suitable for compilation. 

#### Compile

The ``idiom.cc.onnx.compile`` API command invokes the Idiom Compiler, where an ONNX model is compiled for Envise. It accepts mainly three arguments:

* **output_directory** 

    Directory where the output files will get stored after compilation.

* **onnx_file_path**
    
    Path to the ONNX model.
* **batch_size**
    
    Number of samples within a batch used for ONNX export.

In [ ]:
from idiom.cc.onnx import compile
compiled_model_path = 'compiled-onnx'

In [ ]:
print('Starting compiling...')
idiom.cc.onnx.compile(compiled_model_path, onnx_file_name, batch_size=batch_size)
print('Done compiling')

#### Get Dataset

Get the ImageWoof dataset

In [ ]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2-320.tgz

In [ ]:
!tar xf imagewoof2-320.tgz  # silent
!echo "Done"

In [ ]:
!ls -l imagewoof2-320  # check you have the train and validation folders

#### Load Dataset

Load the dataset and initialize the Dataloader features.

In [ ]:
def load_input_data(dataset_path: str):
    print('Loading input data')

    loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(dataset_path, 
                             transforms.Compose([
                                 transforms.Resize((320, 320)),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])),
        batch_size=batch_size, 
        shuffle=False,
        num_workers=1, 
        pin_memory=False)

    data = [{'input': x[0].numpy()} for x in itertools.islice(loader, 1)]

    return data


#### Profile Model

The ``idiom.runtime.profile`` API command invokes the profiler. It measures the model’s performance metrics such as Inferences Per Second (IPS) and latency of your model for Envise by profiling the execution of the model at runtime. 

It mainly accepts two arguments:

* **Compiled Model Directory**

    The Compiled Model Directory where the compilation output resides.

* **Input data**

    A sequence of dictionaries containing model inputs. 

In [ ]:
from idiom.runtime import profile
dataset_path = 'imagewoof2-320'
print('Starting loading & profiling...this may take a few seconds')
input_data = load_input_data(dataset_path)
compiled_model_dir = 'compiled-onnx'
idiom.runtime.profile(compiled_model_dir, input_data, detailed_report=True)

#### Conclusion

This tutorial shows how to validate, compile, and profile a ``ResNet18`` model, and measure its performance metrics for Envise-behavior.
